In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

2025-03-09 15:57:36.408243: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741535859.120609    9297 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741535859.749239    9297 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-09 15:57:47.067928: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
fashion_mnist=tf.keras.datasets.fashion_mnist

(train_images, _), (test_images, _)=fashion_mnist.load_data()

In [1]:
class VAE(tf.keras.Model):
    def __init__(self,latent_dim):
        super(VAE,self).__init__()
        self.latent_dim=latent_dim
        self.encoder=tf.keras.Sequential(
            [
                tf.keras.layers.InputLayer(input_shape=(28,28,1)),
                tf.keras.layers.Conv2D(filters=32,kernel_size=3,strides=(2,2),activation='relu'),
                tf.keras.layers.Conv2D(filters=64,kernel_size=3,strides=(2,2),activation='relu'),
                tf.keras.layers.Flatten(),
                tf.keras.layers.Dense(latent_dim+latent_dim)
            ]
        )
        self.decoder=tf.keras.Sequential(
            [
                tf.keras.layers.InputLayer(input_shape=(latent_dim,)),
                tf.keras.layers.Dense(units=7*7*32,activation='relu'),
                tf.keras.layers.Reshape(target_shape=(7,7,32)),
                tf.keras.layers.Conv2DTranspose(filters=64,kernel_size=3,strides=2,padding='same',activation='relu'),
                tf.keras.layers.Conv2DTranspose(filters=32,kernel_size=3,strides=2,padding='same',activation='relu'),
                tf.keras.layers.Conv2DTranspose(filters=1,kernel_size=3,strides=1,padding='same')
            ]
        )

    def encode(self,x):
        mean,logvar=tf.split(self.encoder(x),num_or_size_splits=2,axis=1)
        return mean,logvar
    
    def reparameterize(self,mean,logvar):
        epsilon=tf.random.normal(shape=mean.shape)
        return epsilon*tf.exp(logvar*0.5)+mean
    
    def sampling(self,mean,logvar):
        epsilon = tf.keras.backend.random_normal(shape=(tf.shape(mean)[0], self.latent_dim), mean=0.0, stddev=1.0)
        return mean + tf.keras.backend.exp(0.5 * logvar) * epsilon

    def decode(self,z):
        logits=self.decoder(z)
        return logits
    
    def forward(self,inputs):
        x=self.encoder(inputs)
        mean,logvar=self.encode(x)
        z=self.reparameterize(mean,logvar)
        reconstructed=self.decoder(z)
        return reconstructed,mean,logvar
    
    def compute_loss(self,x_og,x_recon,mean,logvar):
        recon_loss=tf.keras.losses.binary_crossentropy(tf.keras.backend.flatten(x_og),tf.keras.backend.flatten(x_recon))
        recon_loss=tf.keras.backend.sum(recon_loss)

        kl_loss=-0.5*tf.keras.backend.sum(1+logvar-tf.keras.backend.square(mean)-tf.keras.backend.exp(logvar),axis=-1)

        return tf.keras.backend.mean(recon_loss+kl_loss)
    
    def train_step(self,data):
        with tf.GradientTape() as tape:
            y_pred,mean,logvar=self(data)
            loss=self.compute_loss(data,y_pred,mean,logvar)

        gradients=tape.gradient(loss,self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients,self.trainable_variables))

        return {"loss":loss}

NameError: name 'tf' is not defined

In [ ]:
vae=VAE(latent_dim=16)
vae.compile(optimizers="adam")
vae.fit(train_images,epochs=2)